In [1]:
%matplotlib inline

# Una suave introducción a Autograd

``torch.autograd`` es el motor de diferenciación automática de PyTorch que impulsa el entrenamiento de redes neuronales. En esta sección, obtendrá una comprensión conceptual de cómo autograd ayuda a entrenar una red neuronal.

## Background


Las redes neuronales (NN) son una colección de funciones anidadas que se ejecutan en algunos datos de entrada. Estas funciones están definidas por parámetros (que consisten en pesos y sesgos), que en PyTorch se almacenan en tensores.

El entrenamiento de una NN ocurre en dos pasos:

**Forward Propagation**: en forward prop, la NN hace su mejor conjetura sobre la salida correcta. Ejecuta los datos de entrada a través de cada una de sus funciones para realizar esta conjetura.

**Backward Propagation**: en backprop, la NN ajusta sus parámetros proporcionalmente al error en su conjetura. Lo hace atravesando hacia atrás desde la salida, recopilando las derivadas del error con respecto a los parámetros de las funciones (gradientes) y optimizando los parámetros mediante el descenso de gradiente. Para obtener un tutorial más detallado de backprop, vea este [video](https://www.youtube.com/watch?v=tIeHLnjs5U8) de 3Blue1Brown .

## Uso en Pytorch

Echemos un vistazo a un solo paso de entrenamiento. Para este ejemplo, cargamos un modelo resnet18 previamente entrenado de ``torchvision``. Creamos un tensor de datos aleatorios para representar una sola imagen con 3 canales, y alto y ancho de 64, y su correspondiente ``label`` inicializado a algunos valores aleatorios.

In [2]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/mfnunez/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [01:12<00:00, 644kB/s]


A continuación, ejecutamos los datos de entrada a través del modelo a través de cada una de sus capas para hacer una predicción. Este es el **forward pass**.

In [3]:
prediction = model(data) # forward pass

Usamos la predicción del modelo y la etiqueta correspondiente para calcular el error (`loss`). El siguiente paso es propagar este error a través de la red. La propagación hacia atrás se inicia cuando invocamos `.backward()` en el tensor de error. Luego, Autograd calcula y almacena los gradientes para cada parámetro del modelo en el atributo `.grad` del parámetro.

In [4]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

A continuación, cargamos un optimizador, en este caso SGD con una tasa de aprendizaje de 0.01 y un impulso de 0.9. Registramos todos los parámetros del modelo en el optimizador.

In [5]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Finalmente, llamamos a `.step()` para iniciar el descenso de gradiente. El optimizador ajusta cada parámetro por su gradiente almacenado en `.grad`.

In [6]:
optim.step() #gradient descent

En este punto, tienes todo lo que necesitas para entrenar tu red neuronal. Las secciones a continuación detallan el funcionamiento de autograd; no dude en omitirlas.

--------------




# Diferenciación en Autograd

Echemos un vistazo a cómo `autograd` recopila los degradados. Creamos dos tensores `a` y `b` con `requires_grad=True`. Esto indica autogradque se debe realizar un seguimiento de cada operación en ellos.

In [7]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Creamos otro tensor `Q` a partir de `a` y `b`.

$$
\begin{align}Q = 3a^3 - b^2\end{align}
$$

In [8]:
Q = 3*a**3 - b**2

Supongamos que `a` y `b` son parámetros de una NN, y `Q` es el error. En el entrenamiento de la NN, queremos los gradientes de los parámetros de error, es decir

$$
\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}
$$
$$
\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}
$$

Cuando llamamos a `.backward()` en Q, autograd calcula estos gradientes y los almacena en los respectivos tensores atributo `.grad`.

Necesitamos pasar explícitamente un argumento `gradient` a `Q.backward()` porque es un vector. `gradient` es un tensor de la misma forma que `Q`, y representa el gradiente de `Q` en sí mismo, es decir:

$$
\begin{align}\frac{dQ}{dQ} = 1\end{align}
$$

De manera equivalente, también podemos agregar `Q` en un escalar y llamar hacia atrás implícitamente, como `Q.sum().backward()`.

In [9]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Los degradados ahora se depositan en `a.grad` y `b.grad`

In [10]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


----------------

# Lectura opcional: cálculo vectorial usando autograd

Matemáticamente, si tienes una función con valor vectorial $\vec{y}=f(\vec{x})$, luego el gradiente de $\vec{y}$ con respecto a $\vec{x}$ es una matriz jacobiana $J$:

$$
\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}
$$

en términos generales, `torch.autograd` es un motor de cálculo de producto jacobiano vectorial. Es decir, dado cualquier vector $\vec{v}$, calcula el producto $J^{T}\cdot \vec{v}$

Si \vec{v}$ pasa a ser el gradiente de una función escalar $l=g\left(\vec{y}\right)$:

$$
\begin{align}\vec{v}
   =
   \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}\end{align}
$$

luego, por la regla de la cadena, el producto vectorial-jacobiano sería el gradiente de $l$ con respecto a $\vec{x}$:

$$
\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)\end{align}
$$

Esta característica del producto vector-jacobiano es lo que usamos en el ejemplo anterior; `external_grad` representa $\vec{v}$

--------------------

# Gráfico computacional

Conceptualmente, autograd mantiene un registro de datos (tensores) y todas las operaciones ejecutadas (junto con los nuevos tensores resultantes) en un gráfico acíclico dirigido (DAG) que consta de objetos de [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). En este DAG, las hojas son los tensores de entrada, las raíces son los tensores de salida. Al trazar este gráfico desde las raíces hasta las hojas, puede calcular automáticamente los gradientes usando la regla de la cadena.

En forward pass, autograd hace dos cosas simultáneamente:
 * ejecutar la operación solicitada para calcular un tensor resultante, y
 * mantener la *función de gradiente* de la operación en el DAG.

El backward pass comienza cuando `.backward()` se llama en la raíz del DAG. `autograd` luego:

 * calcula los gradientes de cada uno `.grad_fn`,
 * los acumula en el `.grad` atributo del tensor respectivo, y
 * utilizando la regla de la cadena, se propaga hasta los tensores de las hojas.

A continuación se muestra una representación visual del DAG en nuestro ejemplo. En el gráfico, las flechas están en la dirección del pase hacia adelante. Los nodos representan las funciones hacia atrás de cada operación en el pase hacia adelante. Los nodos de hojas en azul representan nuestros tensores de hojas `a` y `b`.

![dag_autograd](https://pytorch.org/tutorials/_images/dag_autograd.png)

 > **Nota**:
 > **Los DAG son dinámicos en PyTorch**. Una cosa importante a tener en cuenta es que el gráfico se recrea desde cero; después de cada llamada a `.backward()`, autograd comienza a completar un nuevo gráfico. Esto es exactamente lo que le permite utilizar declaraciones de flujo de control en su modelo; puede cambiar la forma, el tamaño y las operaciones en cada iteración si es necesario.

## Exclusión del DAG

`torch.autograd` rastrea las operaciones en todos los tensores que tienen su bandera `requires_grad` establecida en `True`. Para tensores que no requieren gradientes, configurar este atributo `False` para excluirlo del DAG de cálculo de gradiente.

El tensor de salida de una operación requerirá gradientes incluso si solo tiene un tensor de entrada `requires_grad=True`.

In [11]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


En una NN, los parámetros que no calculan gradientes generalmente se denominan **frozen parameters**. Es útil “congelar” parte de su modelo si sabe de antemano que no necesitará los gradientes de esos parámetros (esto ofrece algunos beneficios de rendimiento al reducir los cálculos de autograd).

Otro caso de uso común donde la exclusión del DAG es importante es para ajustar una red previamente capacitada ([finetuning pretrained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html))

En el ajuste fino, congelamos la mayor parte del modelo y, por lo general, solo modificamos las capas del clasificador para hacer predicciones en nuevas etiquetas. Veamos un pequeño ejemplo para demostrarlo. Como antes, cargamos un modelo resnet18 previamente entrenado y congelamos todos los parámetros.

In [12]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Digamos que queremos ajustar el modelo en un nuevo conjunto de datos con 10 etiquetas. En resnet, el clasificador es la última capa lineal `model.fc`. Simplemente podemos reemplazarlo con una nueva capa lineal (descongelada por defecto) que actúa como nuestro clasificador.

In [13]:
model.fc = nn.Linear(512, 10)

Ahora todos los parámetros del modelo, excepto los parámetros de `model.fc`, están congelados. Los únicos parámetros que calculan los gradientes son los pesos y el sesgo de `model.fc`.

In [14]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Observe que, aunque registramos todos los parámetros en el optimizador, los únicos parámetros que calculan gradientes (y, por lo tanto, se actualizan en el descenso de gradientes) son los pesos y el sesgo del clasificador.

La misma funcionalidad de exclusión está disponible como administrador de contexto en [`torch.no_grad()`](https://pytorch.org/docs/stable/generated/torch.no_grad.html)